# Demo

In [1]:
!pip install gradio

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 41.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.7/318.7 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.9/93.9 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 57.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.8/62.8 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.4/71.4 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.2/130.2 kB 8.7 MB/s eta 0:00:00
  Attempting uninstall: tomlkit
    Found existing i

In [2]:
!pip install scikit-learn
!pip install nltk
!pip install transformers

In [3]:
!git clone https://huggingface.co/spaces/sercetexam9/fakenewsdetection

Cloning into 'fakenewsdetection'...
remote: Enumerating objects: 49, done.
remote: Counting objects: 100% (45/45), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 49 (delta 20), reused 0 (delta 0), pack-reused 4 (from 1)
Unpacking objects: 100% (49/49), 17.29 KiB | 393.00 KiB/s, done.
Filtering content: 100% (2/2), 893.35 MiB | 34.77 MiB/s, done.


In [15]:
import re
import string
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from tokenizers import Tokenizer
from wordcloud import WordCloud, STOPWORDS
import gc
nltk.download('punkt')
nltk.download('stopwords')
import pickle
bert_model = pickle.load(open("/content/fakenewsdetection/fakenews_bert.sav", 'rb'))
roberta_model = pickle.load(open("/content/fakenewsdetection/fakenews_roberta.sav", 'rb'))
import gradio as gr
import torch
import time
import torch.nn as nn
from sklearn.metrics import classification_report
import transformers
from transformers import AutoModel, BertTokenizerFast, BertForSequenceClassification,RobertaTokenizer, RobertaForSequenceClassification
import unicodedata
from unicodedata import normalize
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
from sklearn.metrics import accuracy_score

# paramters
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
def wordpre(text):
    """
    Hàm wordpre có tác dụng chuyển tất cả kí tự trong văn bản thành chữ thường, xoá bỏ các kí tự dấu, đường link, dấu câu, dấu cách, xuống dòng,...
    Những kí tự này không có ý nghĩa đáng kể trong việc phân loại tính chất của câu, việc loại bỏ chúng giúp tập trung vào các từ mang ý nghĩa
    quan trọng, giúp đánh giá câu chính xác hơn, ngoài ra còn giảm kích thước từ điển và cải thiện hiệu suất của mô hình.
    """
    text = text.lower()
    # Chuẩn hóa NKFD và loại bỏ dấu
    text = normalize('NFKD', text)
    text = ''.join([c for c in text if not unicodedata.combining(c)])
    text = re.sub('\[.*?\]', '', text)
    text = re.sub("\\W"," ",text) # remove special chars
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text
def tokenize(data):
    """
    nltk.word_tokenize: tách một đoạn văn bản thành các từ riêng biệt.(token hóa) Việc token hóa giúp
    """
    # Tokenization
    data=data.apply(nltk.word_tokenize)
    # Remove stopwords
    stop_words= set(stopwords.words('english'))
    data=data.apply(lambda x: [word for word in x if word not in stop_words])
    return data
def bert_predict(text):
    text=pd.DataFrame([text], columns=["text"])
    text=text["text"]
    text=text.apply(wordpre)
    text=lower_and_tokenize(text)
    nltk.download('wordnet')
    from nltk.corpus import wordnet as wn
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    text = text.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    text = text.apply(lambda x: ' '.join(x))
    # Load the BERT tokenizer
    tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')
    # tokenize and encode sequences in the test set
    max_seq_len=512
    batch_size=16
    tokens_text = tokenizer.batch_encode_plus(
        text.tolist(),
        max_length = max_seq_len,
        padding="max_length",
        truncation=True,
        return_token_type_ids=True,
        add_special_tokens = True,
    )
    # for text set
    text_seq = torch.tensor(tokens_text['input_ids'])
    text_mask = torch.tensor(tokens_text['attention_mask'])
    text_y=torch.tensor([0])
    # wrap tensors
    text_data = TensorDataset(text_seq, text_mask,text_y)

    # sampler for sampling the data during testing
    text_sampler = SequentialSampler(text_data)

    # dataLoader for test set
    text_dataloader = DataLoader(text_data, sampler = text_sampler, batch_size=batch_size)
    seed_val=42
    import random
    import numpy
    random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    y_pred = []
    for batch in text_dataloader:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)
        with torch.no_grad():
            out = bert_model(input_ids, input_mask,labels=labels)
            loss = out[0]
            logi = out.logits
            logits_tensor = torch.tensor(logi)
        pred = torch.argmax(logits_tensor, dim = 1)
        y_pred.append(pred.flatten())

    result=str()
    print(y_pred)
    if y_pred==[1]:
       result="This may be fake news."
    else:
       result="This may be real news."
    return result
def roberta_predict(text):
    text=pd.DataFrame([text], columns=["text"])
    text=text["text"]
    text=text.apply(wordpre)
    text=tokenize(text)
    nltk.download('wordnet')
    from nltk.corpus import wordnet as wn
    from nltk.stem import WordNetLemmatizer
    lemmatizer = WordNetLemmatizer()
    text = text.apply(lambda x: [lemmatizer.lemmatize(word) for word in x])
    text = text.apply(lambda x: ' '.join(x))
    # Load the Roberta tokenizer
    tokenizer = RobertaTokenizer.from_pretrained('roberta-base')
    robert_input_ids=[]
    robert_mask=[]
    max_seq_len = 512
    for sent in text:
        # tokenize and encode sequences in the training set
        tokens_sent = tokenizer.encode_plus(
            sent,
            add_special_tokens = True, # Add '[CLS]' and '[SEP]'
            max_length = max_seq_len,           # Pad & truncate all sentences.
            pad_to_max_length = True,
            return_attention_mask = True,   # Construct attn. masks.
            return_tensors = 'pt',     # Return pytorch tensors.
        )
        robert_input_ids.append(tokens_sent['input_ids'])
        robert_mask.append(tokens_sent['attention_mask'])
    del max_seq_len
    gc.collect()
    # for text set
    text_seq=torch.cat(robert_input_ids,dim=0)
    text_mask=torch.cat(robert_mask,dim=0)
    text_y = torch.tensor([0])
    del robert_input_ids, robert_mask
    gc.collect()
    batch_size=16
    # wrap tensors
    text_data = TensorDataset(text_seq, text_mask, text_y)

    # sampler for sampling the data during testing
    text_sampler = SequentialSampler(text_data)

    # dataLoader for test set
    text_dataloader = DataLoader(text_data, sampler = text_sampler, batch_size=batch_size)
    del text_data
    del text_sampler
    gc.collect()
    seed_val=42
    import random
    import numpy
    random.seed(seed_val)
    torch.manual_seed(seed_val)
    torch.cuda.manual_seed_all(seed_val)
    y_pred = []
    for batch in text_dataloader:
        input_ids = batch[0].to(device)
        input_mask = batch[1].to(device)
        labels = batch[2].to(device)
        with torch.no_grad():
            out = roberta_model(input_ids, input_mask,labels=labels)
            loss = out[0]
            logi = out.logits
            logits_tensor = torch.tensor(logi)
        pred = torch.argmax(logits_tensor, dim = 1)
        y_pred.append(pred.flatten())
    result=str()
    print(y_pred)
    if y_pred==[1]:
       result="This may be fake news."
    else:
       result="This may be real news."
    return result

def predict(text, model_choice):
    if model_choice == "BERT":
      return bert_predict(text)
    elif model_choice == "RoBERTa":
      return roberta_predict(text)
    else:
        raise ValueError("Invalid model choice. Choose either 'BERT' or 'RoBERTa'.")
    return
with gr.Blocks() as demo:
    text_input = gr.Textbox(label="Text", lines=3)
    model_choice = gr.Radio(choices=["BERT", "RoBERTa"], label="Choose Model")
    output = gr.Textbox(label="Predict", lines=1)
    button = gr.Button("Predict")
    button.click(predict, inputs=[text_input, model_choice], outputs=output)
if __name__ == "__main__":
    demo.launch(share=True)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/usr/local/lib/python3.10/dist-packages/torch/storage.py:414: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_onl

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://0ddd966fbd12bc257d.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
